In [1]:
setwd("~/GitHub/ripa-analysis/data")

In [9]:
source('../lib/opp.R')
source('../lib/threshold_test.R')
source('../lib/disparity.R')

Two data files are imported here. The first one is the detailed data collected by LAPD to comply with the state Racial Identity and Profiling Act. 

The second is a data set -- available on the city's data portal -- includes information on the division of the responding officer. The Threshold Test requires a subgeography like a police division to be able to identify differences in search thresholds across an entire city.

In [3]:
# Download large dataset from Dropbox
download.file(
    'https://www.dropbox.com/s/fdpk89gap9mc7ow/RIPA_MASTER_July_April.csv?dl=1', 
    destfile="RIPA_MASTER_July_April.csv")

In [10]:
# Load Data
ripa <- read_csv("RIPA_MASTER_July_April.csv",
col_types = cols(Basis_for_Search = "c"))
colnames(ripa) <- make_ergonomic(colnames(ripa))
lapd <- read_csv("LAPD_online_data_since_July_2018.csv")
colnames(lapd) <- make_ergonomic(colnames(lapd))

To reduce statistical noise, the racial and ethnic groups were categorized into Latino, black, white and "other," which included Asian, Middle Eastern/South Asian, multiracial, Pacific Islander and Native American.

In [6]:
# Filter table to just vehicle stops and divisions
veh_frns <- lapd %>% 
  filter(stop_type == "VEH") %>%
  mutate(division = if_else(
    officer_1_division_number > 0 & officer_1_division_number <= 27,
    division_description_1,
    "OTHER")
  ) %>% 
  select(frn, division, division_description_1, officer_1_division_number) %>% 
  unique()

# The model groups the racial categories into Hispanic, black, white and other
tr_race <- c(
  Latino = "hispanic",
  Black = "black",
  White = "white",
  Asian = "other",
  MiddleEastSouthAsian = "other",
  multiracial = "other",
  `Pacific Islander` = "other",
  `Native American` = "other"
)

This next step joins the two data sets on a unique identifier, "frn," which stands for form reference number. It also creates a hierarchy to flag searches as discretionary or non-discretionary. The filter excludes stops where non-discretionary searches were the primary reason for the police action and categorizes those with multiple reasons for a search. So if a stop included both a consent search and a vehicle inventory search it was included in the analysis because a consent search is considered discretionary and given a higher rank in the hierarchy model.

In [7]:
# Create discretionary search heirarchy 
ripa_veh <- ripa %>% 
  select(frn, race, search, basis_for_search, contraband) %>% 
  filter(frn %in% veh_frns$frn) %>% 
  left_join(veh_frns, by = "frn") %>% 
  mutate(
    search_conducted = search == "TRUE", 
    basis_for_search_single = case_when(
      # Plain view (visible contraband) = 6
      str_detect(basis_for_search, "6") ~ 6,
      # Plain smell (odor of contraband) = 7
      str_detect(basis_for_search, "7") ~ 7,
      # Consent = 1
      str_detect(basis_for_search, "(^1$)|(^1,)|(,1,)|(,1$)") ~ 1,
      # Safety = 2
      str_detect(basis_for_search, "(^2$)|(^2,)|(,2,)|(,2$)") ~ 2,
      # Suspected weapon = 5
      str_detect(basis_for_search, "5") ~ 5,
      # Evidence of crime = 9
      str_detect(basis_for_search, "9") ~ 9,
      # Suspected violation of school policy = 13
      str_detect(basis_for_search, "13") ~ 13,
      # Exigent circumstances/emergency = 11
      str_detect(basis_for_search, "11") ~ 11,
      # K9 detection = 8
      str_detect(basis_for_search, "8") ~ 8,
      # Warrant = 3
      str_detect(basis_for_search, "(^3$)|(^3,)|(,3,)|(,3$)") ~ 3,
      # Probation/parole = 4
      str_detect(basis_for_search, "4") ~ 4,
      # Incident to arrest = 10
      str_detect(basis_for_search, "10") ~ 10,
      # Vehicle inventory = 12
      str_detect(basis_for_search, "12") ~ 12,
      TRUE ~ NA_real_
    ),
    # Non-discretionary searches:
    # 3 = warrant, 4 = as condition of probation/parole, 10 = incident to arrest, 12 = vehicle inventory)
    non_discretionary_search = basis_for_search_single %in% c(3, 4, 10, 12),
    contraband_found = contraband == "TRUE",
    contraband_found = if_else(!search_conducted, FALSE, contraband_found),
    subject_race = as.factor(tr_race[race]),
    sub_geography = division,
    geography = "LA"
  ) 

Now we'll run  two threshold tests -- one for all searches and another for our discretionary search universe. This typically takes about 20 minutes to run. 

Grab a coffee!

In [8]:
# Run threshold test for all searches and for just discretionary searches
tt_results_all_searches <- threshold_test(
  ripa_veh,
  sub_geography,
  geography_col = geography
)
write_rds(tt_results_all_searches, "tt_results_all_searches.rds")

tt_results_discretionary_searches <- threshold_test(
  ripa_veh %>% filter(!non_discretionary_search),
  sub_geography,
  geography_col = geography
)
write_rds(
  tt_results_discretionary_searches, 
  "tt_results_discretionary_searches.rds"
)


# Function wrapper for convergence checks and ppcs
model_checks <- function(model_result) {
  fit <- model_result$metadata$fit
  summary <- summary(fit)$summary
  # Want this to be < 1.05
  print("max Rhat")
  print(summary[,'Rhat'] %>% max(na.rm = T))
  # Want this to be > 0.001
  print("min n_eff")
  print(summary[,'n_eff'] %>% min(na.rm = T))

  search_rate_ppc <- plt_ppc_rates(
    model_result$results$thresholds,
    rstan::extract(model_result$metadata$fit),
    "search_rate",
    numerator_col = n_action,
    denominator_col = n,
    title = str_c("LA threshold ppc - search rates")
  )

  hit_rate_ppc <- plt_ppc_rates(
    model_result$results$thresholds,
    rstan::extract(model_result$metadata$fit),
    "hit_rate",
    numerator_col = n_outcome,
    denominator_col = n_action,
    title = str_c("LA threshold ppc - hit rates")
  )

  list(
    search_rate_ppc = search_rate_ppc,
    hit_rate_ppc = hit_rate_ppc
  )
}

all_search_checks <- model_checks(tt_results_all_searches)
disc_search_checks <- model_checks(tt_results_discretionary_searches)

tt_results_all_searches$results$aggregate_thresholds
tt_results_discretionary_searches$results$aggregate_thresholds

Warning message:
"2.33% of data was null for required columns and removed"Warning message in metadata["stan_warnings"] <- summary(warnings()):
"number of items to replace is not a multiple of replacement length"Warning message:
"2.38% of data was null for required columns and removed"recompiling to avoid crashing R session
Warning message in metadata["stan_warnings"] <- summary(warnings()):
"number of items to replace is not a multiple of replacement length"

[1] "max Rhat"
[1] 1.004415
[1] "min n_eff"
[1] 1400.747
[1] "Weighted RMS prediction error: 0.04%"
[1] "Weighted RMS prediction error: 4.99%"
[1] "max Rhat"
[1] 1.005524
[1] "min n_eff"
[1] 1211.272
[1] "Weighted RMS prediction error: 0.05%"
[1] "Weighted RMS prediction error: 7.80%"


race,avg_threshold,threshold_ci,threshold_diff,diff_ci
black,15.91%,"(15.33%, 16.48%)",-2.05%,"(-3.50%, -0.64%)"
hispanic,15.38%,"(14.85%, 15.93%)",-2.58%,"(-4.01%, -1.19%)"
other,15.00%,"(13.48%, 16.61%)",-2.97%,"(-4.97%, -0.90%)"
white,17.96%,"(16.68%, 19.28%)",,


race,avg_threshold,threshold_ci,threshold_diff,diff_ci
black,18.49%,"(17.60%, 19.40%)",-5.79%,"(-8.86%, -3.02%)"
hispanic,18.91%,"(18.01%, 19.87%)",-5.38%,"(-8.42%, -2.58%)"
other,20.50%,"(17.83%, 23.49%)",-3.78%,"(-7.77%, 0.26%)"
white,24.28%,"(21.64%, 27.21%)",,


These result tables show that black and Latino vehicle occupants had significantly lower search thresholds than whites for all searches and for discretionary searches.